<a href="https://colab.research.google.com/github/EddyGiusepe/Cyber_Security_1/blob/main/CyberSecurity_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h2 align='center'>Cyber Security -- Kaggle</h2> 


**Data Scientist Jr.:**  Dr.Eddy Giusepe Chirinos Isidro


Neste Script reproduziremos e exploraremos ataques cibernéticos. Basicamente temos Dados onde no último mês, alguns eventos de SEGURANÇA CIBERNÉTICA com durações e padrões diferentes foram registrados.


Nesse sentido, podemos derivar:

* Endereço de IP de destino mais direcionado,
* A maioria das portas lógicas atacadas,
* Tipo de ataque mais frequente/comum,
* Hora diferente do dia (odd (estranho), horas, dia ou noite),
* Encontrar um padrão.

Este Script está baseado nos seguintes links:

* [Cyber Security](https://www.kaggle.com/iamranjann/exploring-attacks-cybersec/notebook).
* [Cyber Security no YouTube](https://www.youtube.com/watch?v=7q8XQSUeUPc&list=PLlUZLZydkS79ctBEkAqeNj5k0RNvAvvD7&index=1).

## Importamos as nossas bibliotecas

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipaddress
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from datetime import datetime, timedelta
import math
import missingno as msno
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore') 

## Carregamos nossos Dados de Cyber Security

In [2]:
df = pd.read_csv("/content/drive/MyDrive/4_EDDY_ISH_TECNOLOGIA/3_Data_Science_Augusto_Eddy/Attacks CyberSecurity Kaggle/Cybersecurity_attacks.csv")


In [3]:
df.shape

(178031, 11)

In [4]:
df.head(5)

,Attack category,Attack subcategory,Protocol,Source IP,Source Port,Destination IP,Destination Port,Attack Name,Attack Reference,.,Time
0,Reconnaissance,HTTP,tcp,175.45.176.0,13284,149.171.126.16,80,Domino Web Server Database Access: /doladmin.n...,-,.,1421927414-1421927416
1,Exploits,Unix 'r' Service,udp,175.45.176.3,21223,149.171.126.18,32780,Solaris rwalld Format String Vulnerability (ht...,CVE 2002-0573 (http://cve.mitre.org/cgi-bin/cv...,.,1421927415-1421927415
2,Exploits,Browser,tcp,175.45.176.2,23357,149.171.126.16,80,Windows Metafile (WMF) SetAbortProc() Code Exe...,CVE 2005-4560 (http://cve.mitre.org/cgi-bin/cv...,.,1421927416-1421927416
3,Exploits,Miscellaneous Batch,tcp,175.45.176.2,13792,149.171.126.16,5555,HP Data Protector Backup (https://strikecenter...,CVE 2011-1729 (http://cve.mitre.org/cgi-bin/cv...,.,1421927417-1421927417
4,Exploits,Cisco IOS,tcp,175.45.176.2,26939,149.171.126.10,80,Cisco IOS HTTP Authentication Bypass Level 64 ...,CVE 2001-0537 (http://cve.mitre.org/cgi-bin/cv...,.,1421927418-1421927418


In [5]:
# Observamos que as nossas colunas são:

df.columns

Index(['Attack category', 'Attack subcategory', 'Protocol', 'Source IP',
       'Source Port', 'Destination IP', 'Destination Port', 'Attack Name',
       'Attack Reference', '.', 'Time'],
      dtype='object')